# Defining a system of differential equations

The starting point, when using PyGOM to study a system of differential equations (DE's), is to encapsulate the relevant information into a class {class}`.SimulateOde`.
Once defined as an instance of this class, the system is ready for the application of PyGOM's various features such as simulation and parameter fitting.

```{note}
Whilst the following section is quite long and involves some mathematics, we recommend not skipping it since there can be subtleties when it comes to correctly defining a model and these need to be precisely communicated to PyGOM.
```

## Continuous time Markov chains

We first introduce a particular kind of system called a continuous time Markov chain (CTMC).
These commonly appear in the study of chemically reacting systems and epidemiology, where the quantities of interest typically refer to counts of something (be that molecules or people) in different compartments.
In general, and this will be the case we consider here, the variables may not only refer to discrete quantities but could take continuous values too.

In a CTMC, $n$ variables, $\mathbf{y}(t) = \left(y_{1}(t),\ldots,y_{n}(t)\right)$, describe the state of the system at time, $t$.
There are $T$ different types of event (which PyGOM refers to as **Transitions**) and the effect of an event occurring is to change the state by some amount.
Mathematically speaking, if event $j$ occurs then the variable $y_i$ changes by $\mathcal{D}_{ij}$, which may depend on the current state, time and parameters, $\boldsymbol{\theta} = \left(\theta_{1},\ldots, \theta_{p}\right)$:

$$\begin{aligned}
\Delta y_i \text{ due to event of transition type, j } = \mathcal{D}_{ij}(\mathbf{y}, \boldsymbol{\theta}, t)
\end{aligned}$$

where $\mathcal{D}$ is a $n \times T$ matrix.
These events occur randomly in time and independently of each other at rates (also possibly dependent on the state of the system):

$$\begin{aligned}
\boldsymbol{\lambda}(\mathbf{y}, \boldsymbol{\theta}, t) = \left(\lambda_{1}(\mathbf{y}, \boldsymbol{\theta}, t),\ldots, \lambda_{T}(\mathbf{y}, \boldsymbol{\theta}, t)\right)
\end{aligned}$$

To demonstrate how this works with an example, consider a Susceptible-Infected-Recovered (SIR) epidemic model.
```{note}
For those unfamiliar with the SIR model, an outline can be found in the {doc}`common models <common_models/SIR>` section.
```
In this case, the state of the system is specified by 3 variables: $S$, $I$ and $R$.
There are 2 types of event, infection and recovery, which occur at rates $\frac{\beta S I}{N}$ and $\gamma I$ respectively.
The result of an infection event happening is to reduce $S$ by 1, increment $I$ by 1 and leave $R$ unchanged. 
Recovery reduces $I$ by 1 and increments $R$ by 1, leaving $S$ unchanged.
This information can be encapsulated in the framework above by the objects $\mathbf{y}$, $\boldsymbol{\lambda}$ and $\mathcal{D}$ as:

$$\begin{aligned}
\mathbf{y}=
\begin{pmatrix}
S\\
I\\
R
\end{pmatrix}
\text{, } \boldsymbol{\lambda} =
\begin{pmatrix}
\frac{\beta S I}{N}\\
\gamma I
\end{pmatrix}
\text{, } \mathcal{D} =
\begin{pmatrix}
-1 & 0\\
1 & -1\\
0 & 1
\end{pmatrix}
\text{, } \boldsymbol{\theta} =
\Big( \beta, \gamma, N\Big)
\end{aligned}$$

To fully specify the CTMC, this must be accompanied by a master equation which describes the probability that the system moves from the current state to the next.
We omit this information here and instead just note that the system evolves according to the two independent stochastic processes: infection and recovery.
Our SIR example is a special case in which the events involve transitions of individuals from one compartment to another, thus conserving the total number of individuals (note that the column sums of $\mathcal{D}$ are all 0).
However, in general:

- Each event need not effect only 2 variables.
- Each event doesn't have to change variables by $\pm 1$, or even by a constant amount.
- Variables don't have to be discrete and/or positive.

For example, let's introduce the variable, $E$, which tracks the net economic impact of the epidemic (negative for loss and positive for gain).
Let's assume that each infection costs a time dependent amount, $c_i=c_0 e^{-\frac{t}{\tau}}$, so that treatment becomes less expensive with time. In this case:

- An infection event now causes a change in 3 variables ($S$, $I$ and $E$). These are correlated, despite not having direct flows between all of them.
- Upon an infection, $E$ decreases by a time varying, continuous amount.
- $E$ is not discrete and will take a negative value if any infections occur.

The final kinds of processes we should account for are births and deaths.
If individuals are born in a susceptible state at a rate proportional to the total fixed population, $\mu N$, this represents a flow of people with no explicit origin in the model.
Similarly, deaths occurring in each compartment ($S$, $I$, $R$) at rates ($\mu S$, $\mu I$, $\mu R$) are flows with no explicit destination (unless we add an ``afterlife'' compartment).
Just to increase the scope of our example model, let's further assume (rather unrealistically) that each birth contributes an amount $c_b$ to the economy.
Including the economic modelling plus births and deaths in our model:

$$\begin{aligned}
\mathbf{y}=
\begin{pmatrix}
S\\
I\\
R\\
E
\end{pmatrix}
\text{, } \boldsymbol{\lambda} =
\begin{pmatrix}
\frac{\beta S I}{N}\\
\gamma I\\
\mu N\\
\mu S\\
\mu I\\
\mu R
\end{pmatrix}
\text{, } \mathcal{D} =
\begin{pmatrix}
-1 & 0 & 1 & -1 & 0 & 0\\
1 & -1 & 0 & 0 & -1 & 0\\
0 & 1 & 0 & 0 & 0 & -1\\
-c_i(t) & 0 & c_b & 0 & 0 & 0
\end{pmatrix}
\text{, } \boldsymbol{\theta} =
\Big( \beta, \gamma, N, \mu, c_0, \tau, c_b, \mu \Big)
\end{aligned}$$

It is worth noting that in the infection process (first column of $\mathcal{D}$), the pair of states, $S$ and $I$, do still have a special status in that they are involved in a ``true'' transition of individuals.
This is also true for the recovery process (second column of $\mathcal{D}$), where people flow between $I$ and $R$. 
We can further claim a similar status for births and deaths, which represent real flows in and out of the system.
The other changes which occur during an event might be viewed as correlated side-effects (which PyGOM will call **secondary effects**).
We can preserve this information by separating the matrix $\mathcal{D}$ into the explicit origins, $\mathbf{o}$, destinations, $\mathbf{d}$ (where $o_k$ and $d_k$ are the respective origin and destination of transition $k$) and secondary effects, $\mathcal{S}$: 

$$\begin{aligned}
\mathbf{y}=
\begin{pmatrix}
S\\
I\\
R\\
E
\end{pmatrix}
\text{, } \boldsymbol{\lambda} =
\begin{pmatrix}
\frac{\beta S I}{N}\\
\gamma I\\
\mu N\\
\mu S\\
\mu I\\
\mu R
\end{pmatrix}
\text{, } \mathbf{o}=
\begin{pmatrix}
S\\
I\\
\text{None}\\
S\\
I\\
R
\end{pmatrix}
\text{, } \mathbf{d}=
\begin{pmatrix}
I\\
R\\
S\\
\text{None}\\
\text{None}\\
\text{None}
\end{pmatrix}
\text{, } \mathcal{S} =
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
-c_i(t) & 0 & c_b & 0 & 0 & 0
\end{pmatrix}
\text{, } \boldsymbol{\theta} =
\Big( \beta, \gamma, N, \mu, c_0, \tau, c_b, \mu \Big)
\end{aligned}$$

Note that the "None" indicates that births have no origin and deaths no destination.
How PyGOM can solve for the time evolution of such a set of stochastic differential equations (SDE's) is a subject for later in this documentation [LINK].
The focus of the current section is how to successfully communicate all of the above details to PyGOM.
Before we get to this, we now consider the case where knowledge of the underlying transitions is either not necessary, or the system is not characterised by transitions in the first place.

### Ordinary Differential Equations

If we instead wish to study how the SIR (plus $E$) model evolves on average and are in a regime where non-integer population counts are not an issue, we can write a set of equations:

$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\frac{\beta SI}{N} + \mu (N - S) \\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \frac{\beta SI}{N} - (\gamma+\mu) I \\
\frac{\mathrm{d} R}{\mathrm{d} t} &= (\gamma-\mu) I \\
\frac{\mathrm{d} E}{\mathrm{d} t} &= c_b \mu S - c_i(t) \frac{\beta SI}{N}
\end{aligned}$$

These are examples of a type of equation known as Ordinary Differential Equations (ODE's):
a relationship between quantities and their rates of change with respect to one independent variable - in this case, time.
In general, a set of ODE's has the form:

$$\begin{aligned}
\frac{\mathrm{d} \mathbf{y}}{\mathrm{d} t} = f(\mathbf{y},\boldsymbol{\theta}, t)
\end{aligned}$$

where $\mathbf{y}$ and $\boldsymbol{\theta}$ denote dependent variables and parameters respectively, as previously, and $f$ is a vector function giving the rates of change of each dependent variable.
In converting the CTMC model, these equations can be found via:

$$\begin{aligned}
\frac{\mathrm{d} y_i}{\mathrm{d} t}=f_i=\sum_{\text{transitions, }j} \mathcal{D}_{ij}\lambda_j
\end{aligned}$$

Therefore, the familiar set of ODE's describing the SIR model are, in fact, an approximation of an underlying CTMC.
However, not all ODE's are necessarily the result of a CTMC.
In the table below we provide 2 examples of models which can be described through and ODE or a CTMC perspective and 2 models which only have an ODE interpretation:

| Name | ODE | CTMC |
|----|---------|:------------------------:|
| Hooke's law | $\begin{aligned} \frac{\mathrm{d}x}{\mathrm{d}t}&=v\\ \frac{\mathrm{d}v}{\mathrm{d}t}&= -\frac{k}{m}x \end{aligned}$ | NA|
| Newton's law of cooling | $\begin{aligned} \frac{\mathrm{d}T}{\mathrm{d}t}=-\frac{1}{\tau} \left( T-T_{\text{env}} \right) \end{aligned}$ | NA|
| Lotka-Volterra | $\begin{aligned} \frac{\mathrm{d} n_1}{\mathrm{d}t}&=\alpha n_1 - \beta n_1 n_2\\ \frac{\mathrm{d}n_2}{\mathrm{d}t}&= -\gamma n_2 + \delta n_1 n_2 \end{aligned}$ | $\begin{aligned} \boldsymbol{\lambda} = \begin{pmatrix} \alpha n_1\\ \beta n_1 n_2 \\ \gamma n_2 \\ \delta n_1 n_2 \end{pmatrix} \text{, } \mathcal{D} = \begin{pmatrix} 1 & -1 & 0 & 0\\ 0 & 0 & -1 & 1 \end{pmatrix} \end{aligned}$             |
| SIR | $\begin{aligned} \frac{\mathrm{d} S}{\mathrm{d}t}&=-\frac{\beta S I}{N}\\ \frac{\mathrm{d}I}{\mathrm{d}t}&= \frac{\beta S I}{N} - \gamma I\\ \frac{\mathrm{d}R}{\mathrm{d}t}&= \gamma I \end{aligned}$ | $\begin{aligned} \boldsymbol{\lambda} = \begin{pmatrix} \frac{\beta S I}{N}\\ \gamma I \end{pmatrix} \text{, } \mathcal{D} = \begin{pmatrix} -1 & 0\\ 1 & -1\\ 0 & 1 \end{pmatrix} \end{aligned}$ |

The first, Hooke's law, describes the dynamics the position, $x$, and velocity, $v$, of a mass attached to a spring.
Whilst it is possible to introduce stochasticity to this system, perhaps due to random fluctuations in the spring properties, this will not take the form of a CTMC.
There is actually nothing to mathematically prevent one from modelling the system as a CTMC with $\boldsymbol{\lambda}= \begin{pmatrix} 1\\ 1 \end{pmatrix}$, $\mathcal{D}=\begin{pmatrix} v & 0 \\ 0 & -\frac{k}{m} x \end{pmatrix}$, the result just has no basis in reality.
The same story applies for Newton's law of cooling, where an object which is warmer than the environment approaches thermal equilibrium with its surroundings.

As we have seen, an SIR is amenable to both a CTMC or an ODE approach.
Also included in the table is the Lotka-Volterra predator-prey model where we see that a CTMC implementation acts through 2 birth and 2 death processes.

## ODE vs CTMC Definition in PyGOM

PyGOM allows the user flexibility in choosing which of these perspectives to use, or indeed combine if possible, in order to build their models.
If stochastic solutions are required, however, then PyGOM *must* know the underlying transitions.
The easiest way is to supply this information directly, however, {doc}`in some cases <unroll/unrollSimple>`, it can also attempt to reverse engineer transitions if ODE's are instead supplied.
If stochastic solutions are not required (or, indeed, the system has no underlying CTMC), then the system can be specified either in terms of ODE's or transitions (if possible).

One advantage of specifying our system using the transition approach is that many will find transitions a more intuitive language when thinking about compartmental models.
Perhaps more beneficial, though, is that when building models with transitions using PyGOM, it enables the computer to do our book-keeping when converting transitions to ODE equations.
This reduces the error of, for example, including a flow out of one state, but forgetting to include it in the recipient state.
For example, in the case above, when indicating infections with $-\frac{\beta SI}{N}$ in the $\frac{\mathrm{d}S}{\mathrm{d}t}$ equation, we had to remember to include the opposite sign in the $\frac{\mathrm{d}I}{\mathrm{d}t}$ equation.

## Defining the model with PyGOM

Defining the system is handled by feeding {class}`.Transition` objects into the central class, {class}`.SimulateOde`.
An instance of the {class}`.Transition` class takes one of four types, which are specified by the `transition_type` argument, letting {class}`.SimulateOde` know which type of information it is being given.
This type could be `Transition` or `ODE`, as we've just been discussing, or one of two other available options, namely `Birth` and `Death` processes, which are essentially of the transition `Transition` type too, but without a destination state.
The {class}`.Transition` class accepts 3 or 4 parameters depending on which type is being defined, as summarised in the below table:

|                 |          Transition             |            Equation               |                  Birth                 |                  Death                 |
|:---------------:|:-------------------------------:|:---------------------------------:|:--------------------------------------:|:--------------------------------------:|
| transition_type | T                               | ODE                               | B                                      | D                                      |
| origin          | $o_k$ | State equation pertains to, $y_i$ | State birth process populates, $y_i$   | State death process depletes, $y_i$    |
| destination     | $d_k$  | n/a                               | n/a                                    | n/a                                    |
| equation        | Rate of transition, $\lambda_k$ | $\frac{dy_i}{dt}$                 | Birth rate, $\lambda_k$ | Death rate, $\lambda_k$ |
| secondary_effects | $\mathcal{S}_k$                       | n/a                 | $\mathcal{S}_k$  | $\mathcal{S}_k$  |

```{note}
Arguably, the state which the birth process populates could be defined as a destination rather than an origin, but this is not the convention adopted here.

Furthermore, the {class}`.Transition` class may have been better named.
One of the four possible input types is also called "transition", which would incorrectly imply that it has a special place within the class, given that it shares its name.
```

Let's see how this is implemented for our example SIR(E) system without births and deaths.
We import the relevant modules and then define our transitions:

# Defining with pygom

Depends on if you want to solve ODE's. Indeed PyGOM can be used for non-ode systems.

## Simple SIR

By the book we should do:

In [ ]:
from pygom import Event, Transition, SimulateOde

#TODO: Birth/Death/Between is now obvious from origin/destination

# 1) Infection
## Transition
transition_infection=Transition(origin='S', destination='I', transition_type='T', magnitude='1')
## Event
event_infection=Event(transition_list=[transition_infection],
                      rate='beta*S*I/N')

# 2) Recovery
## Transition
transition_recovery=Transition(origin='I', destination='R', transition_type='T', magnitude='1')
## Event
event_recovery=Event(transition_list=transition_recovery,
                     rate='gamma*I')

# We specify states and parameters
params=['beta', 'gamma', 'N']
states=[('S', (0, None)), ('I', (0, None)), ('R', (0, None))]

model=SimulateOde(state=states, param=params, event=[event_infection, event_recovery])

model.get_ode_eqn()

Since it is fairly common to have change $\pm 1$ and only one transition involved in an Event

We can actually get away with:

In [ ]:
transition_infection=Transition(origin='S', destination='I', transition_type='T', equation='beta*S*I/N')
transition_recovery=Transition(origin='I', destination='R', transition_type='T', equation='gamma*I')

# We specify states and parameters
params=['beta', 'gamma', 'N']
states=[('S', (0, None)), ('I', (0, None)), ('R', (0, None))]

model=SimulateOde(state=states, param=params, event=[transition_infection, transition_recovery])

model.get_ode_eqn()

In [1]:
from pygom import Transition, TransitionType

# Define SIR model through a list of ODEs
# odeList = [
#     Transition(transition_type=TransitionType.ODE, origin='S', equation='-beta*S*I/N + mu*N - mu*S'),
#     Transition(transition_type=TransitionType.ODE, origin='I', equation='beta*S*I/N - gamma*I - mu*I'),
#     Transition(transition_type=TransitionType.ODE, origin='R', equation='gamma*I- mu*I') 
# ]

odeList = [
    Transition(transition_type=TransitionType.ODE, origin='S', equation='-beta*S*I/N'),
    Transition(transition_type=TransitionType.ODE, origin='I', equation='beta*S*I/N - gamma*I'),
    Transition(transition_type=TransitionType.ODE, origin='R', equation='gamma*I'),
    Transition(transition_type=TransitionType.ODE, origin='E', equation='-ci*beta*S*I/N') 
]

# Define SIR model through a list of transitions
transList = [
    Transition(transition_type=TransitionType.T, origin='S', destination='I', equation='beta*S*I/N', secondary_effects=[('E', '-ci')]),
    Transition(transition_type=TransitionType.T, origin='I', destination='R', equation='gamma*I')
]

We now initialise two {class}`.SimulateOde` objects using these different approaches.
In addition to the ODE or transition information, we must specify which variables are parameters and which refer to states:

In [2]:
stateList = ['S', 'I', 'R', 'E']
paramList = ['beta', 'gamma', 'N', 'ci']

We import the relevant module and define the model in two different ways:

In [3]:
from pygom import SimulateOde
model_ode = SimulateOde(state=stateList, param=paramList, ode=odeList)           # model defined via equations
model_tra = SimulateOde(state=stateList, param=paramList, transition=transList)  # model defined via transitions

We can use the {func}`.get_ode_eqn` function to verify that the models are equivalent:

In [4]:
model_ode.get_ode_eqn()

Matrix([
[          -I*S*beta/N],
[-I*gamma + I*S*beta/N],
[              I*gamma],
[       -I*S*beta*ci/N]])

In [5]:
model_tra.get_ode_eqn()

Matrix([
[          -I*S*beta/N],
[-I*gamma + I*S*beta/N],
[              I*gamma],
[       -I*S*beta*ci/N]])

## Modifying models

Once an instance of {class}`.SimulateOde` has been defined it may be modified.
For example, let's now add birth and death processes to the the previously defined SIR model of `model_ode`.
We must update the parameters of the class to reflect any additions:

In [6]:
model_tra.param_list = model_tra.param_list + ['mu', 'cb']

We then include the additional processes, where we assume that all new births enter the population as susceptibles:

In [7]:
birthDeathList = [Transition(origin='S', equation='mu*N', transition_type=TransitionType.B, secondary_effects=[('E', 'cb')]),  
                  Transition(origin='S', equation='mu*S', transition_type=TransitionType.D), 
                  Transition(origin='I', equation='mu*I', transition_type=TransitionType.D), 
                  Transition(origin='R', equation='mu*R', transition_type=TransitionType.D)]

model_tra.birth_death_list = birthDeathList

We can again use {func}`.get_ode_eqn` to verify that the equations are updated correctly:

In [9]:
model_tra.get_ode_eqn()

Matrix([
[   -I*S*beta/N + N*mu - S*mu],
[-I*gamma - I*mu + I*S*beta/N],
[              I*gamma - R*mu],
[              -I*S*beta*ci/N]])

In [ ]:
class Transition:
    rate='beta*S*I'
    type='Transition'
    origin='S'
    destination='I'
    secondary_effects=[('I_tot', '1'), ('B', 'i_cost')]




class Transition:
    type='Transition'
    origin='S'
    destination='I'

class Transition:
    type='birth'
    destination='I_tot'
    change='1'

class Transition:
    type='death'
    origin='B'
    change='i_cost'

class Event:
    rate='beta*S*I'
    transitions=[T1, T2, T3]